<h1>Decision Tree</h1>

To achieve this result, I referenced several online tutorials and our in class lecture. Many of the tutorials I found on this subject were complete and working examples of how to implement a CART Decision Tree from scratch, but disliked the data format or were written in an older version of Python (usually Python 2). Therefore, I had quite a bit of trouble even making something functional. I've included another (failed) attempt in addition to this successful one that I couldn't seem to make work.

For some reason a few tutorials called the node class "Tree" with a capital T and then the completed tree "tree" with a lowercase. I found this confusing so I changed it in my implementation. 

In [1]:
import math

In [2]:
  class Node:
    def __init__(self, parent=None):
        self.parent = parent
        self.children = []
        self.label = None
        self.classCounts = None
        self.splitFeatureValue = None
        self.splitFeature = None

In [3]:
def printBinaryDecisionTree(root, indentation=""):
   if root.children == []:
      print( "%s%s, %s %s" % (indentation, root.splitFeatureValue, root.label, root.classCounts))
   else:
      printBinaryDecisionTree(root.children[0], indentation + "\t")

      if indentation == "": # processing the root
         print( "%s%s" % (indentation, root.splitFeature))
      else:
         print( "%s%s, %s" % (indentation, root.splitFeatureValue, root.splitFeature))

      if len(root.children) == 2:
         printBinaryDecisionTree(root.children[1], indentation + "\t")

In [4]:
def dataToDistribution(data):
    allLabels = [label for (point, label) in data]
    numEntries = len(allLabels)
    possibleLabels = set(allLabels)

    dist = []
    for aLabel in possibleLabels:
        dist.append(float(allLabels.count(aLabel)) / numEntries)
    return dist

In [5]:
def gini(dist):
    return -sum([p * math.log(p, 2) for p in dist])

In [6]:
def splitData(data, featureIndex):
   # get possible values of the given feature
    attrValues = [point[featureIndex] for (point, label) in data]

    for aValue in set(attrValues):
      # compute the piece of the split corresponding to the chosen value
        dataSubset = [(point, label) for (point, label) in data
                    if point[featureIndex] == aValue]

        yield dataSubset

In [7]:
def gain(data, featureIndex):
    
    entropyGain = gini(dataToDistribution(data))

    for dataSubset in splitData(data, featureIndex):
          entropyGain -= gini(dataToDistribution(dataSubset))

    return entropyGain

In [8]:
def homogeneous(data):

    return len(set([label for (point, label) in data])) <= 1


In [9]:
def majorityVote(data, node):
    labels = [label for (pt, label) in data]
    choice = max(set(labels), key=labels.count)
    node.label = choice
    node.classCounts = dict([(label, labels.count(label)) for label in set(labels)])
    return node

In [10]:
def buildDecisionTree(data, root, remainingFeatures):

    if homogeneous(data):
        root.label = data[0][1]
        root.classCounts = {root.label: len(data)}
        return root

    if len(remainingFeatures) == 0:
        return majorityVote(data, root)

   # find the index of the best feature to split on
    bestFeature = max(remainingFeatures, key=lambda index: gain(data, index))

    if gain(data, bestFeature) == 0:
        return majorityVote(data, root)

    root.splitFeature = bestFeature

   # add child nodes and process recursively
    for dataSubset in splitData(data, bestFeature):
        aChild = Node(parent=root)
        aChild.splitFeatureValue = dataSubset[0][0][bestFeature]
        root.children.append(aChild)

        buildDecisionTree(dataSubset, aChild, remainingFeatures - set([bestFeature]))

    return root

In [11]:
def decisionTree(data):
    return buildDecisionTree(data, Node(), set(range(len(data[0][0]))))

In [12]:
def classify(tree, point):

    if tree.children == []:
        return tree.label
    else:
        matchingChildren = [child for child in tree.children
            if child.splitFeatureValue == point[tree.splitFeature]]

    if len(matchingChildren) == 0:
         raise Exception("Classify is not able to handle noisy data. Use classify2 instead.")

    return classify(matchingChildren[0], point)

In [13]:
def dictionarySum(*dicts):
    sumDict = {}
    for aDict in dicts:
          for key in aDict:
            if key in sumDict:
                sumDict[key] += aDict[key]
            else:
                sumDict[key] = aDict[key]

    return sumDict

In [14]:
def classifyNoisy(tree, point):
   if tree.children == []:
      return tree.classCounts
   elif point[tree.splitFeature] == '?':
      dicts = [classifyNoisy(child, point) for child in tree.children]
      return dictionarySum(*dicts)
   else:
      matchingChildren = [child for child in tree.children
         if child.splitFeatureValue == point[tree.splitFeature]]

      return classifyNoisy(matchingChildren[0], point)

In [15]:
def classify2(tree, point):
   counts = classifyNoisy(tree, point)

   if len(sorted(counts)) == 1:
      return sorted(counts)[0]
   else:
      return max(counts, key=lambda k: counts[k])

In [16]:
def testClassification(data, tree, classifier=classify2):
   actualLabels = [label for point, label in data]
   predictedLabels = [classifier(tree, point) for point, label in data]

   correctLabels = [(1 if a == b else 0) for a,b in zip(actualLabels, predictedLabels)]
   return float(sum(correctLabels)) / len(actualLabels)

In [17]:
def testTreeSize(noisyData, cleanData):
   import random

   for i in range(1, len(cleanData)):
      tree = decisionTree(random.sample(cleanData, i))
      print( str(testClassification(noisyData, tree)) + ", ",)

In [18]:
if __name__ == '__main__':
   with open('house-votes-84.csv', 'r') as inputFile:
      lines = inputFile.readlines()

   data = [line.strip().split(',') for line in lines]
   data = [(x[1:], x[0]) for x in data]

   cleanData = [x for x in data if '?' not in x[0]]
   noisyData = [x for x in data if '?' in x[0]]
    
   tree = decisionTree(cleanData)
   print (classify(tree, ['y' for _ in range(len(data[0][0]))]))
   print (classify(tree, ['n' for _ in range(len(data[0][0]))]))

republican
democrat


Error with 'clean' data (abstaining votes removed from dataset):

In [19]:
testClassification(cleanData,tree,classifier=classify)

0.9871244635193133

Error with 'noisy' data:

In [20]:
testClassification(data,tree,classifier=classify2)

0.963302752293578